# What happens:
File is created and email is prepared

## Import 

In [3]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import win32com.client as win32

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [4]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/Repo - close-out Currency Check"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

### Make connection with Oracle DB

In [5]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save selection
### Also see how much time it took

In [ ]:
%%time
d = pd.read_sql_query("""select *
from fdw_netting_Agreement
where netting_type = 'Netting Repo'
and close_out_currency is null
""", connection_RAY)

Wall time: 46 ms


### Save var in an excel file

In [ ]:
# determining the name of the file
file_name = rd + '_missing_coc.xlsx'

# saving the excel
d.to_excel(file_name, index=False)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


### Send the email 

In [ ]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'rudolf.sellinger@rbinternational.com'
mail.Cc = 'markus.hochleitner@rbinternational.com; eugen.puseizer@rbinternational.com;'
mail.Subject = 'Repo - Missing close-out currency deals (' + rd + ')'
mail.Body = '''Dear Redolf,

please find attached the deals with missing close-out currency, kindly asking to check whether correct.
If not, please provide us the correct close-out currency and fix it in the source system.

Kr
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

# To attach a file to the email (optional):
attachment  = directory + "/" + file_name
mail.Attachments.Add(attachment)

mail.Display()